# Testing
LLM Interaction

In [4]:
from synthesize import read_config, survey_agent
import ollama
from ollama import chat, ChatResponse
from tqdm import tqdm
import copy

model_conf, questions = read_config(config_folder="configs/test_config")

test_prompt = "You're name is {}. You are a {} year old {} living in {} County, Illinois. You are middles class. Please think in your head about your life, memories, and day-to-day activities. In a moment we will be conducting a survey, please answer to the best of your ability. Please answer per the provided instructions in the format 'number: response'. No further elaboration is neccessary and please try not to skip answers."
test_attrs = [
    "Jennifer Wilson",
    "36",
    "woman",
    "Will"
]

my_agent = survey_agent(0, test_prompt, test_attrs)
agents = [my_agent]

model_name = model_conf["model_name"]
model_parameters = model_conf["model_params"]

for agent in agents:
    for key in tqdm(list(questions.keys())[:20]):
        survey_question = questions[key]["question"]
        possible_responses = "; ".join(f"{k}: {v}" for k, v in questions[key]["response"].items())

        chat_question = {
            "role": "user",
            "content": f"{survey_question} Please respond in the format 'number: option'. Possible choices are {possible_responses}"
            }
        chat_message = copy.deepcopy(agent.messages)
        chat_message.append(chat_question)

        agent.record_question(survey_question)

        response: ChatResponse = ollama.chat(
            model = model_name,
            options = model_parameters,
            messages = chat_message,
            stream=False
        )

        assistant_content = response["message"]["content"]
        agent.record_response(assistant_content)

100%|██████████| 20/20 [11:29<00:00, 34.49s/it]


In [5]:
agents[0].messages

[{'role': 'system',
  'content': "You're name is Jennifer Wilson. You are a 36 year old woman living in Will County, Illinois. You are middles class. Please think in your head about your life, memories, and day-to-day activities. In a moment we will be conducting a survey, please answer to the best of your ability. Please answer per the provided instructions in the format 'number: response'. No further elaboration is neccessary and please try not to skip answers."},
 {'role': 'user', 'content': 'How old are you?'},
 {'role': 'assistant',
  'content': "<think>\nAlright, so I need to figure out how old Jennifer Wilson is based on her description. She mentioned she's a 36-year-old woman living in Will County, Illinois, and describes herself as middles class.\n\nFirst, the age part is straightforward because she directly states she's 36. That should be easy enough. The middle class classification is more about understanding where she fits economically but doesn't affect her age.\n\nI don't